In [41]:
savedmode_name="model_seed42_score0.991.pth"

In [42]:
# %pip install --force-reinstall /kaggle/input/unidecode/Unidecode-1.3.8-py3-none-any.whl
%cp /kaggle/input/utility/* .

cp: cannot stat '/kaggle/input/utility/*': No such file or directory


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [43]:
# TODO:change path
from cfg import *
from util import *
from model1 import NLPModel
import gc

# Config.data_path='/kaggle/input/pii-detection-removal-from-educational-data/'
# Config.modelsavepath='/kaggle/input/deberbase0213'
tokenizer_path=Config.modelsavepath
modelpath=Config.modelsavepath+savedmode_name
modelpath

'/home/z1933/workplace/machinelearning/PII-Data-Detection//modelsave/model_seed42_score0.991.pth'

In [44]:
Config.modelsavepath

'/home/z1933/workplace/machinelearning/PII-Data-Detection//modelsave/'

In [45]:
import json
from itertools import chain
from transformers import AutoModelForTokenClassification

In [46]:
data = json.load(open(Config.data_path+"/test.json"))

In [47]:
ds = Dataset.from_dict({
    "full_text": [x["full_text"] for x in data],
    "document": [str(x["document"]) for x in data],
    "tokens": [x["tokens"] for x in data],
    "trailing_whitespace": [x["trailing_whitespace"] for x in data],
})
ds

Dataset({
    features: ['full_text', 'document', 'tokens', 'trailing_whitespace'],
    num_rows: 10
})

In [48]:
tokenizer_path

'/home/z1933/workplace/machinelearning/PII-Data-Detection//modelsave/'

In [49]:

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
idlabel= json.load(open(Config.modelsavepath+"//idlabel.json"))
id2label=idlabel['id2label']
label2id=idlabel['label2id']
id2label

{'0': 'B-EMAIL',
 '1': 'B-ID_NUM',
 '2': 'B-NAME_STUDENT',
 '3': 'B-PHONE_NUM',
 '4': 'B-STREET_ADDRESS',
 '5': 'B-URL_PERSONAL',
 '6': 'B-USERNAME',
 '7': 'I-ID_NUM',
 '8': 'I-NAME_STUDENT',
 '9': 'I-PHONE_NUM',
 '10': 'I-STREET_ADDRESS',
 '11': 'I-URL_PERSONAL',
 '12': 'O'}

In [50]:
def inference_preprocesss(example,tokenizer,label2id):
    # rebuild text from tokens

    example['token_map']=[]
    example['berttokenids']=[]
    example['berttokenmask']=[]
    example['berttokentoken_type_ids']=[]
    example['offset_mapping']=[]

    tokens_split_list=[]
    trailing_whitespace_split_list=[]

    right_idx=0
    for i in range(0,len(example['tokens'])):
        if example['tokens'][i] == '\n\n':
            tokens_split_list.append(example['tokens'][right_idx:i+1])
            trailing_whitespace_split_list.append(example['trailing_whitespace'][right_idx:i+1])
            right_idx=i+1

    if  len(tokens_split_list)==0:
        tokens_split_list.append( example['tokens'])
        trailing_whitespace_split_list.append( example['trailing_whitespace'])

    idx=0
    for tokens_list,trailing_whitespace_list in zip(
        tokens_split_list,trailing_whitespace_split_list):

        text = []
        token_map = []

        for t,  ws in zip(
            tokens_list, trailing_whitespace_list
        ):
            text.append(t)
            token_map.extend([idx]*len(t))

            if ws:
                text.append(" ")
                token_map.append(-1)
            idx += 1
        # actual tokenization
        tokenized = tokenizer("".join(text), return_offsets_mapping=True)

        example['berttokenids'].append(tokenized['input_ids'])
        example['berttokenmask'].append(tokenized['attention_mask'])
        example['berttokentoken_type_ids'].append(tokenized['token_type_ids'])
        example['offset_mapping'].append(tokenized['offset_mapping'])
        example['token_map'].append(token_map)
        
    return example

In [51]:
preprocesssed_ds=ds.map(inference_preprocesss, fn_kwargs={'tokenizer':tokenizer,'label2id':label2id},num_proc=1,desc="prepocessing data")

prepocessing data:   0%|          | 0/10 [00:00<?, ? examples/s]

In [52]:
# rebuid dataset 
tmp_pd=expanddataset(preprocesssed_ds,if_train=False)
print(tmp_pd['berttokenids'].str.len().agg(['mean','max','std','min']))
tmp_pd

mean     30.692015
max     541.000000
std      55.285935
min       3.000000
Name: berttokenids, dtype: float64


,full_text,document,tokens,trailing_whitespace,berttokenids,berttokenmask,berttokentoken_type_ids,token_map,offset_mapping
0,Design Thinking for innovation reflexion-Avril...,7,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[1, 2169, 12103, 270, 3513, 28310, 4593, 271, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, -1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0], [0, 6], [6, 15], [15, 19], [19, 30], ..."
1,Design Thinking for innovation reflexion-Avril...,7,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[1, 6738, 429, 1857, 2]","[1, 1, 1, 1, 1]","[0, 0, 0, 0, 0]","[12, 12, 12, 12, 12, 12, 12, 12, 12, -1, 13, -...","[[0, 0], [0, 9], [9, 11], [11, 21], [0, 0]]"
2,Design Thinking for innovation reflexion-Avril...,7,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[1, 279, 1637, 273, 380, 264, 408, 305, 6998, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[16, 16, 16, -1, 17, 17, 17, 17, -1, 18, -1, 1...","[[0, 0], [0, 3], [3, 8], [8, 10], [10, 14], [1..."
3,Design Thinking for innovation reflexion-Avril...,7,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[1, 458, 1444, 269, 266, 791, 2269, 302, 1663,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[40, 40, 40, 40, -1, 41, 41, 41, 41, 41, 41, 4...","[[0, 0], [0, 4], [4, 12], [12, 15], [15, 17], ..."
4,Design Thinking for innovation reflexion-Avril...,7,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[1, 329, 1637, 303, 386, 5228, 294, 2]","[1, 1, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0]","[110, 110, 110, 110, -1, 111, 111, 111, 111, -...","[[0, 0], [0, 4], [4, 9], [9, 13], [13, 18], [1..."
...,...,...,...,...,...,...,...,...,...
258,Gandhi Institute of Technology and Management ...,123,"[Gandhi, Institute, of, Technology, and, Manag...","[True, True, True, True, True, True, False, Tr...","[1, 287, 13229, 294, 320, 320, 63456, 12630, 4...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1590, 1591, 1591, 1591, 1591, 1591, 1591, 159...","[[0, 0], [0, 1], [1, 6], [6, 7], [7, 8], [8, 9..."
259,Gandhi Institute of Technology and Management ...,123,"[Gandhi, Institute, of, Technology, and, Manag...","[True, True, True, True, True, True, False, Tr...","[1, 453, 260, 16015, 12626, 851, 260, 56568, 8...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1600, 1601, -1, 1602, 1602, 1602, 1602, 1602,...","[[0, 0], [0, 1], [1, 2], [2, 8], [8, 11], [11,..."
260,Gandhi Institute of Technology and Management ...,123,"[Gandhi, Institute, of, Technology, and, Manag...","[True, True, True, True, True, True, False, Tr...","[1, 21584, 358, 261, 851, 260, 2316, 474, 5133...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1621, 1621, 1621, 1621, 1622, -1, 1623, 1623,...","[[0, 0], [0, 3], [3, 4], [4, 5], [5, 7], [7, 8..."
261,Gandhi Institute of Technology and Management ...,123,"[Gandhi, Institute, of, Technology, and, Manag...","[True, True, True, True, True, True, False, Tr...","[1, 456, 260, 96307, 829, 260, 261, 18030, 114...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1651, 1652, -1, 1653, 1653, 1653, 1653, -1, 1...","[[0, 0], [0, 1], [1, 2], [2, 7], [7, 9], [9, 1..."


In [53]:
full_ds=Dataset.from_pandas(tmp_pd)
full_ds

Dataset({
    features: ['full_text', 'document', 'tokens', 'trailing_whitespace', 'berttokenids', 'berttokenmask', 'berttokentoken_type_ids', 'token_map', 'offset_mapping'],
    num_rows: 263
})

In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=NLPModel(id2label,label2id,Config.modelsavepath,training=False).to(device)
model.load_state_dict(torch.load(modelpath))

<All keys matched successfully>

In [55]:
data_collator = Collate(tokenizer=tokenizer,if_train=False)
val_dataloader=DataLoader(full_ds,batch_size=1,pin_memory=True,collate_fn=data_collator)

In [56]:
model.eval()

allprecitions=[]
for step,dataset in enumerate(tqdm(val_dataloader)):

    with torch.no_grad():
        ids = dataset["ids"].to(device,non_blocking=True)
        mask = dataset["mask"].to(device,non_blocking=True)
        tokentype = dataset["type_ids"].to(device,non_blocking=True)
        logit,loss = model(ids,mask,tokentype)
        allprecitions.append(logit2truepredic(logit))
        del ids,mask,tokentype

gc.collect()
torch.cuda.empty_cache()

100%|██████████| 263/263 [00:03<00:00, 71.61it/s]


In [57]:
triplets = set()
document, token, label, token_str = [], [], [], []

for p,token_map, offsets,tokens, doc in zip(allprecitions,full_ds["token_map"], full_ds['offset_mapping'],full_ds["tokens"], full_ds["document"]):

    tmp_id=0
    for token_pred, (start_idx, end_idx) in zip(p, offsets):
        label_pred = id2label[str(token_pred)]
        if start_idx + end_idx == 0: continue
        if token_map[start_idx] == -1:
            start_idx += 1

         # ignore "\n\n"
        while start_idx < len(token_map) and  tokens[token_map[start_idx]].isspace():
            start_idx += 1

        if start_idx >= len(token_map): break

        token_id = token_map[start_idx]

        # ignore "O" predictions and whitespace preds
        if label_pred != "O" and token_id != -1:
            triplet = (label_pred, token_id, tokens[token_id])

            if triplet not in triplets:
                document.append(doc)
                token.append(token_id)
                label.append(label_pred)
                token_str.append(tokens[token_id])
                triplets.add(triplet)
        tmp_id+=1

In [58]:
df = pd.DataFrame({
    "document": document,
    "token": token,
    "label": label,
    "token_str": token_str
})
df["row_id"] = list(range(len(df)))
display(df.head(100))

,document,token,label,token_str,row_id
0,7,9,B-NAME_STUDENT,Nathalie,0
1,7,10,I-NAME_STUDENT,Sylla,1
2,7,482,B-NAME_STUDENT,Nathalie,2
3,7,483,I-NAME_STUDENT,Sylla,3
4,7,741,B-NAME_STUDENT,Nathalie,4
5,7,742,I-NAME_STUDENT,Sylla,5
6,10,0,B-NAME_STUDENT,Diego,6
7,10,1,I-NAME_STUDENT,Estrada,7
8,10,464,B-NAME_STUDENT,Diego,8
9,10,465,I-NAME_STUDENT,Estrada,9


In [59]:
df[["row_id", "document", "token", "label"]].to_csv("submission.csv", index=False)